In [19]:
import sys
import os

# Add src to the system path to allow imports from the src directory
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, '../../..', 'src'))
sys.path.insert(0, src_path)

In [20]:
from dolfin import *
import ufl
from materials import *
from params import *
from matplotlib import pyplot as plt
%matplotlib inline

## Initialize problem

In [21]:
params = Params()
params.print_table()

| Parameter    |        Value | Unit   |
|--------------|--------------|--------|
| dimension    |      3       | -      |
| plane_stress |      0       | Bool   |
| Lx           |    500       | m      |
| Ly           |    750       | m      |
| Lz           |    125       | m      |
| g            |      9.81    | m/s²   |
| rho_sea      |   1020       | N/m³   |
| E0_ice       |      9.5e+09 | Pa     |
| nu_ice       |      0.35    | -      |
| rho_i_ice    |    917       | N/m³   |
| sigma_c_ice  | 118500       | Pa     |
| ci           |      1       | -      |
| l            |      0.625   | m      |


In [22]:
g = params.g
rho_sea = params.rho_sea
E0_ice = params.E0_ice
nu_ice = params.nu_ice
rho_i_ice = params.rho_i_ice
sigma_c_ice = params.sigma_c_ice

In [23]:
xdmf = XDMFFile("output/static/single_crevasse_3D.xdmf")
xdmf.parameters["functions_share_mesh"] = True
xdmf.parameters["rewrite_function_mesh"] = False
xdmf.parameters["flush_output"] = True

## Initialize Mesh

In [24]:
Lx, Ly, Lz = 500, 750, 125
mesh = Mesh()
with XDMFFile("mesh/single_crevasse/non-adaptive/mesh.xdmf") as infile:
    infile.read(mesh)

mesh = refine(mesh)
print(f"Mesh minimum cell size (hmin): {mesh.hmin()}, Number of cells: {mesh.num_cells()}")


Mesh minimum cell size (hmin): 0.7834976600929583, Number of cells: 10111448


## Initialize material model

In [25]:
ice = LinearElastic(mesh, params)

# Initialize boundary conditions

In [26]:
FDG0 = FunctionSpace(mesh, "DG", 0)
TDG0 = TensorFunctionSpace(mesh, "DG", 0)
V1 = VectorFunctionSpace(mesh, "CG", 1)

In [27]:
front = CompiledSubDomain("near(x[1], 0)")
back = CompiledSubDomain("near(x[1], L)",L=Ly)
left = CompiledSubDomain("near(x[0], 0)")
right_csd = CompiledSubDomain("near(x[0], B)", B=Lx)
bottom = CompiledSubDomain("near(x[2], 0)")
top = CompiledSubDomain("near(x[2], H)", H=Lz)

bottom_roller = DirichletBC(V1.sub(2), Constant(0), bottom)
left_roller = DirichletBC(V1.sub(0), Constant(0), left)
front_roller = DirichletBC(V1.sub(1), Constant(0), front)
back_roller = DirichletBC(V1.sub(1), Constant(0), back)
bc_u = [bottom_roller, left_roller,front_roller,back_roller]

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
right_csd.mark(boundaries, 1)

ds = Measure("ds", subdomain_data=boundaries)

## Setup and Solve Problem

In [28]:
u, v = TrialFunction(V1), TestFunction(V1)
u_sol = Function(V1, name="disp")
stress_xx, cdf = Function(FDG0, name="sigma_xx"), Function(FDG0, name="cdf")

In [29]:
t = Expression((0,"(h - x[2] >= 0 ?-rho_H2O * grav*(h - x[2]) : 0)", 0), h=125/2, rho_H2O=rho_sea, grav= params.g, degree=1)

# t = Constant((0, 0,0))
f = Constant((0, 0,-ice.rho * params.g))

In [30]:
a = (1+1e-4)*inner(ice.stress(u), ice.strain(v)) * dx
L = dot(f, v) * dx + dot(t, v) * ds(1)

In [31]:
solve(a == L, u_sol, bc_u, solver_parameters={"linear_solver": "gmres", "preconditioner": "ilu"})

Solving linear variational problem.


## Post Processing

In [32]:
stress_xx.assign(project(ice.principal_stress(u_sol)[0,0]/1e6, FDG0)) # N/m2 -> MPa
cdf.assign(project(ice.crack_driving_force(u_sol), FDG0))

In [33]:
xdmf.write(u_sol,0)
xdmf.write(stress_xx,0)
xdmf.write(cdf,0)
xdmf.close()

In [34]:
cdf.vector().max()

85.85282705876577

In [35]:
stress_xx.vector().max()

1.1041507572073868

In [36]:
u_sol.vector().max(), u_sol.vector().min()

(0.018938457985338043, -0.011663000524427153)